# 03 — TaskFlow: API Gateway, Roteamento e Perfis (local vs docker)

**Objetivo:** adicionar o **API Gateway** (Fase 1) e explicar como roteamos `/api/tasks/**` → `/tasks/**`, alternando entre **ambiente local** e **Docker Compose** sem editar ficheiros.

## Dependências (Gateway)
No `gateway/pom.xml` usar o artefacto **não deprecado**:

```xml
<dependency>
  <groupId>org.springframework.cloud</groupId>
  <artifactId>spring-cloud-starter-gateway-server-webflux</artifactId>
</dependency>
```


## Configuração (chaves novas) e variável `TASKS_BASE_URL`

`gateway/src/main/resources/application.yml`

```yaml
server:
  port: 8080

spring:
  application:
    name: gateway

  cloud:
    gateway:
      server:
        webflux:
          routes:
            - id: tasks
              # falls back to localhost if env var is not defined
              uri: "${TASKS_BASE_URL:http://localhost:8081}"
              predicates:
                - Path=/api/tasks/**
              filters:
                - StripPrefix=1
          default-filters:
            - DedupeResponseHeader=Access-Control-Allow-Credentials Access-Control-Allow-Origin
```


## Alternar entre ambientes

- **Local (STS)**: não definas `TASKS_BASE_URL` → usa `http://localhost:8081` (fallback).
- **Docker Compose**: define no serviço `gateway`:
```yaml
environment:
  - TASKS_BASE_URL=http://tasks-service:8081
```

### Ver rotas carregadas (diagnóstico)
Ativa temporariamente no `application.yml` do gateway:
```yaml
management:
  endpoints:
    web:
      exposure:
        include: health,info,gateway
  endpoint:
    gateway:
      enabled: true
```
Depois: `http://localhost:8080/actuator/gateway/routes`.


## Testes por *curl* (via Gateway)

```bash
# via gateway
curl -s http://localhost:8080/api/tasks | jq

# criar tarefa via gateway
curl -s -H "Content-Type: application/json" \
  -d '{"title":"Task via Gateway","description":"demo","projectId":"P1","assignee":"Ana"}' \
  http://localhost:8080/api/tasks | jq
```

> Dica: se te enganares e chamares `http://localhost:8081/api/tasks`, verás 404 (Whitelabel) — esse é o backend sem gateway.
